In [2]:
import requests, csv
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from collections import OrderedDict
import tqdm
from datetime import datetime, date, timedelta
import time
from urllib.request import Request, urlopen
from urllib.error import HTTPError
from pathlib import Path
import math
from typing import Final

In [2]:
head = {
      'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
      'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
      'Accept-Encoding': 'none',
      'Accept-Language': 'fr;q=0.9,en-US,en;q=0.8',
      'Connection': 'keep-alive'}


In [230]:
team_dict = pd.read_csv("team_letter.csv")

In [4]:
def stat_creation(stats, bs):
    global id_ext, id_dom, dnp_ext, mp_dom, dnp_dom
    final_dict = dict()
    for stat in stats:
        scrap = bs.findAll('td',{"data-stat":f"{stat}"})
        scrap = [int(text.text) for text in scrap]
        ext = [scrap[i] for i in range(id_ext)] + dnp_ext * [0]
        dom = [scrap[id_dom-1-i] for i in range(len(mp_dom))][::-1] + dnp_dom * [0]
        final_dict[stat] = ext + dom
    return final_dict

In [5]:
stat_list = ["fg","fga","fg3","fg3a","ft","fta","orb","drb","trb","ast",
             "stl","blk","tov","pf"]

# Scrapping Function

In [6]:
def game_scrap(bs:Final[BeautifulSoup], code:str, ot:str, season:int) -> pd.Datadrame:

    global id_ext, id_dom, dnp_ext, mp_dom, dnp_dom
    # Check the scores
    score = bs.findAll('div',{"class":"scores"})
    score = [int((text.text).strip("\n")) for text in score]
    ext_score, dom_score = score[0], score[1]

    # Get player names
    player = bs.findAll('th',{"class":"left"})
    players = [text.text for text in player]
    team_tot_idx = [i for i in range(len(players)) if players[i] == "Team Totals"]
    player_code = [str(text).split('csv="')[1].split('" data')[0] for text in player[:team_tot_idx[0]] + player[team_tot_idx[-2]+1:team_tot_idx[-1]]]
    player_link = ["https://www.basketball-reference.com"+str(text).split('href="')[1].split('">')[0] \
               for text in player[:team_tot_idx[0]] + player[team_tot_idx[-2]+1:team_tot_idx[-1]]]
    names = players[:team_tot_idx[0]] + players[team_tot_idx[-2]+1:team_tot_idx[-1]]
    team_ext_name, team_dom_name = players[:team_tot_idx[0]], players[team_tot_idx[-2]+1:team_tot_idx[-1]]

    # Get team names
    get_teams = bs.findAll("a")
    final_team = []
    
    for team in get_teams:
        if team.has_attr('href') and team.text.find("Schedule") !=-1 and team.text.find("Schedules") ==-1 \
        and team.text.find("Results") ==-1 and team.text.split(" Sched")[0] not in final_team:
            final_team.append(team.text.split(" Sched")[0])

    team_list = len(team_ext_name) * [final_team[0]] + len(team_dom_name) * [final_team[1]]
    abrev_list = len(team_ext_name) * [team_dict[final_team[0]]] + len(team_dom_name) * [team_dict[final_team[1]]]

    # Get minutes played
    mp = bs.findAll('td',{"data-stat":"mp"})
    mp = [text.text for text in mp]
    if pd.isna(ot):
        min_240 = [i for i in range(len(mp)) if mp[i] == "240"]
        min_120 = [i for i in range(len(mp)) if mp[i] == "120"]
    elif ot == "OT":
        min_240 = [i for i in range(len(mp)) if mp[i] == "265"]
        min_120 = [i for i in range(len(mp)) if mp[i] == "25"]
    else :
        n = int(ot.split("OT")[0])
        min_240 = [i for i in range(len(mp)) if mp[i] == str(240+n*25)]
        min_120 = [i for i in range(len(mp)) if mp[i] == "25"]

    mp_ext, mp_dom = mp[:min_240[0]], mp[min_120[-1]+1:min_240[-1]]

    dnp_ext = len(team_ext_name) - len(mp_ext)
    dnp_dom = len(team_dom_name) - len(mp_dom)
    min_list = mp_ext + dnp_ext * [0] + mp_dom + dnp_dom * [0]

    field = ["Away"]*len(team_ext_name) + ["Home"]*len(team_dom_name)

    status = ["starter"]*5 + ["subs"]*(len(team_ext_name) - dnp_ext - 5) + ["dnp"]*dnp_ext + \
             ["starter"]*5 + ["subs"]*(len(team_dom_name) - dnp_dom - 5) + ["dnp"]*dnp_dom

    # Get points scored
    pt = bs.findAll('td',{"data-stat":"pts"})
    pt = [int(text.text) for text in pt]
    id_ext, id_dom = pt.index(ext_score), pt.index(dom_score)
    ext_pt = [pt[i] for i in range(id_ext)]
    dom_pt = [pt[id_dom-1-i] for i in range(len(mp_dom))][::-1]
    pt_list = ext_pt + dnp_ext * [0] + dom_pt + dnp_dom * [0]

    # Get other stats
    stat_list = ["fg","fga","fg3","fg3a","ft","fta","orb","drb","trb","ast","stl","blk","tov","pf"]
    other_stats = stat_creation(stat_list, bs)

    game_code = [abrev_list[0]+abrev_list[-1]+code[:8]] * len(team_list)
    date = [code[:4] + "-" + code[4:6] + "-" + code[6:8]] * len(team_list)
    game_score = [str(ext_score) + "-" + str(dom_score)] * len(team_list)
    target = [1 if dom_score > ext_score else 0] * len(team_list)

    df = pd.DataFrame(other_stats)
    df.insert(0,"team", team_list)
    df.insert(1,"abrev", abrev_list)
    df.insert(2,"field", field)
    df.insert(3,"name", names)
    df.insert(4,"codes", player_code)
    df.insert(5,"status", status)
    df.insert(6,"mp", mp_ext + dnp_ext * [0] + mp_dom + dnp_dom * [0])
    df.insert(7,"pts", ext_pt + dnp_ext * [0] + dom_pt + dnp_dom * [0])
    df.insert(10,"fg_pct", round(df["fg"]/df["fga"],3))
    df.insert(13,"fg3_pct", round(df["fg3"]/df["fg3a"],3))
    df.insert(16,"ft_pct", round(df["ft"]/df["fta"],3))

    df["game_code"] = game_code
    df["date"] = date
    df["season"] = season
    df["score"] = game_score
    df["OT"] = ot
    df["target"] = target

    return df


In [81]:
code_df = pd.read_excel("game_codes_2025.xlsx")

In [8]:
columns = ['team', 'abrev', 'field', 'name', 'codes', 'status', 'sp', 'pts', 'fg', 'fga',
       'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb',
       'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'game_code', 'date', 'season','score','ot'
       'target']

In [82]:
df = pd.DataFrame(columns = columns)

In [83]:
n = 1

In [84]:
for idx, row in tqdm.tqdm(code_df.iterrows(), total=len(code_df)):
    try:
        r1 = Request(f"https://www.basketball-reference.com/boxscores/{row["game_codes"]}.html", headers=head)
        webpage1 = urlopen(r1).read()
        html = webpage1.decode("utf-8", errors="replace")
        bs = BeautifulSoup(html, "html.parser")
        df = pd.concat([df, game_scrap(bs, row["game_codes"], row["Overtime"], row["season"])])
        
        if idx%100 == 0:
            df.to_csv(f"csvs/total_games{n}.csv")
            df = pd.DataFrame(columns = columns)
            n+=1

    except HTTPError as e:
        if e.code == 404:
            print(f"404 pour {code}, on passe")
            time.sleep(2)
            continue
        else:
            print(f"HTTPError {e.code} : {e.reason}")
    time.sleep(2)

C:\Users\lione\AppData\Local\Temp\ipykernel_24552\16627551.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, game_scrap(bs, row["game_codes"], row["Overtime"])])
C:\Users\lione\AppData\Local\Temp\ipykernel_24552\16627551.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, game_scrap(bs, row["game_codes"], row["Overtime"])])
C:\Users\lione\AppData\Local\Temp\ipykernel_24552\16627551.py:7: FutureWarning: The behavior of DataFrame concatenation with empty o

In [89]:
path = Path(r"C:\Users\lione\Desktop\codes\Projet-NBA\csvs")

files = [f for f in path.iterdir() if f.is_file()]

In [90]:
ids = sorted([int(file.name.split("games")[1].split(".csv")[0]) for file in files])

In [91]:
new_df = pd.DataFrame(columns = columns)

In [92]:
for id in ids:
    for file in files:
        if str(id) in file.name:
            use_file = pd.read_csv(file, index_col=0)
            new_df = pd.concat([new_df, use_file])

C:\Users\lione\AppData\Local\Temp\ipykernel_24552\2903977046.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, use_file])


In [ ]:
new_df = pd.concat([new_df, df])

In [ ]:
# Change time values in seconds to avoid the hour notation

In [94]:
new_df["sp"] = (
    new_df["sp"]
    .astype(str)
    .replace("0", "0:00")
    .str.split(":")
    .apply(lambda x: int(x[0]) * 60 + int(x[1]))
)

In [ ]:
new_df["status"] = new_df["status"].replace("dnp","tdnp")

In [95]:
new_df.to_excel("total_games.xlsx")

In [103]:
player_codes = list(new_df["codes"].unique())

In [104]:
def player_scrap(code):
    code = code
    r = Request(url="https://www.basketball-reference.com/players/"+code[0]+"/"+code+".html", headers = head)
    webpage = urlopen(r).read()
    bs = BeautifulSoup(webpage, "html.parser")

    try :
        raw_pos = bs.findAll("p")[1].text.split("\n\n  Position:\n  \n  ")[1].split("\n")[0]
        
        if raw_pos.find(",") != -1:
            pos_list = raw_pos.split(", ")[0:2]
        else : 
            pos_list = raw_pos.split(" and ")
            
        height = int(bs.findAll("p")[2].text.split("(")[1].split("cm")[0])
        weight = int(bs.findAll("p")[2].text.split("cm,\xa0")[1].split("kg")[0])

    except IndexError :
        try:
            raw_pos = bs.findAll("p")[2].text.split("\n\n  Position:\n  \n  ")[1].split("\n")[0]

            if raw_pos.find(",") != -1:
                pos_list = raw_pos.split(", ")[0:2]
            else : 
                pos_list = raw_pos.split(" and ")

            height = int(bs.findAll("p")[3].text.split("(")[1].split("cm")[0])
            weight = int(bs.findAll("p")[3].text.split("cm,\xa0")[1].split("kg")[0])

        except IndexError :
            try:
                raw_pos = bs.findAll("p")[3].text.split("\n\n  Position:\n  \n  ")[1].split("\n")[0]

                if raw_pos.find(",") != -1:
                    pos_list = raw_pos.split(", ")[0:2]
                else : 
                    pos_list = raw_pos.split(" and ")

                height = int(bs.findAll("p")[4].text.split("(")[1].split("cm")[0])
                weight = int(bs.findAll("p")[4].text.split("cm,\xa0")[1].split("kg")[0])
                
            except IndexError :
                raw_pos = bs.findAll("p")[4].text.split("\n\n  Position:\n  \n  ")[1].split("\n")[0]

                if raw_pos.find(",") != -1:
                    pos_list = raw_pos.split(", ")[0:2]
                else : 
                    pos_list = raw_pos.split(" and ")

                height = int(bs.findAll("p")[5].text.split("(")[1].split("cm")[0])
                weight = int(bs.findAll("p")[5].text.split("cm,\xa0")[1].split("kg")[0])

    return code, pos_list, height, weight

In [108]:
players = []

In [109]:
for code in tqdm.tqdm(player_codes):
    players.append(player_scrap(code))
    time.sleep(2) # important to avoid too many requests error

100%|████████████████████████████████████████| 109/109 [04:54<00:00,  2.70s/it]


In [110]:
player_df = pd.DataFrame(players, columns=["code", "position", "height", "weight"])

In [119]:
abr_pos = {"Power Forward":["PF",4], "Shooting Guard":["SG",2],	"Small Forward":["SF",3], "Center":["C",5], "Point Guard":["PG",1]}

In [120]:
player_df["pos_1"] = player_df["position"].apply(lambda x : abr_pos.get(x[0], [None])[0])

In [121]:
player_df["pos_2"] = player_df["position"].apply(lambda x : abr_pos.get(x[1], [None])[0] if len(x) > 1 else "")

In [127]:
player_df["num_pos_1"] = player_df["position"].apply(lambda x : abr_pos.get(x[0], [None])[1] if x[0] in abr_pos.keys() else "")

In [128]:
player_df["num_pos_2"] = player_df["position"].apply(lambda x : abr_pos.get(x[1], [None])[1] if len(x) > 1 else "")

In [114]:
player_df.to_excel("player_table.xlsx")

# Merge tables and organize data

In [131]:
games = pd.read_excel("total_games.xlsx")

In [132]:
players = pd.read_excel("player_table.xlsx")

In [141]:
games = games.merge(
    players[[
        "code",
        "height",
        "weight",
        "pos_1",
        "pos_2",
        "num_pos_1",
        "num_pos_2"
    ]],
    left_on="codes",
    right_on="code",
    how="left"
)

In [147]:
games = games.drop("code", axis=1)

In [154]:
games["sub_pos"] = games.apply(lambda x: "BC" if (x["num_pos_1"] < 4) and (x["status"] == "subs") 
                               else "FC" if (x["num_pos_1"] > 3) and (x["status"] == "subs")
                               else "", axis=1)

C:\Users\lione\AppData\Local\Temp\ipykernel_24552\1134113797.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games["sub_pos"] = games.apply(lambda x: "BC" if (x["num_pos_1"] < 4) and (x["status"] == "subs")


In [149]:
cols_to_move = [
    "height",
    "weight",
    "pos_1",
    "pos_2",
    "num_pos_1",
    "num_pos_2",
    "sub_pos"
]

insert_at = 5

cols = [c for c in games.columns if c not in cols_to_move]

new_order = (
    cols[:insert_at]
    + cols_to_move
    + cols[insert_at:]
)

games = games[new_order]

In [160]:
games = games.sort_values(
    by=["date", "game_code", "field","status", "num_pos_1", "height"]
)

In [165]:
games.to_excel("ordered_games.xlsx")

# Create order for starters

In [3]:
games = pd.read_excel("ordered_games.xlsx")

In [158]:
active_player = games[games["status"] != "tdnp"]

In [159]:
# Add the "real" order of the starters to the sub_post (starters have been sorted but sometimes, we still had the same number for different posts)

In [160]:
col = active_player["sub_pos"]

mask = col.isna()  # Put True everytime the value is a NaN in the serie

In [161]:
# mask.shift() will shift the values to get the previous one (adding a NaN to the beginning of the serie)
# mask.ne(mask.shift()) will create a Serie stating when mask is not equal (ne) to mask.shift() allowing to point a change of group
# NaN to BC/FC, BC/FC to Nan
# cumsum() will create group id at each change of type of data 
# cumcount() will count the number of iteration per group starting by 0 (reason of the + 1)

In [162]:
group = (mask & (mask.ne(mask.shift()) | active_player["field"].ne(active_player["field"].shift()))).cumsum()

In [163]:
active_player.loc[mask, "sub_pos"] = (
    mask.groupby(group).cumcount() + 1
)

In [164]:
num_cols = [
    "height", "weight", "sp","pts",'fg', 'fga', 'fg3', 'fg3a',
    'ft', 'fta','orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf'
]

In [165]:
info_cols = ['team', 'abrev','date', 'season', 'score', 'OT', 'target']

In [166]:
agg_df = (
    active_player.groupby(['game_code','field','sub_pos'], as_index=False)[num_cols]
      .mean()
      .round(1)
)

In [167]:
agg_df = agg_df.merge(
    active_player[info_cols + ['game_code','field','sub_pos']].drop_duplicates(),
    on=['game_code','field','sub_pos'],
    how='left'
)

In [168]:
agg_df = agg_df.sort_values(by=['season','date','game_code','field','sub_pos']).reset_index(drop=True)

### Version with a normalization by the seconds played

In [169]:
stat_col = ["pts", "fg", "fga", "fg3", "fg3a","ft", "fta", "orb", "drb", "ast","stl", "blk", "tov", "pf"]

In [170]:
agg_df_time = agg_df.copy()

In [171]:
agg_df_time[stat_col] = agg_df_time[stat_col].div(agg_df_time["sp"], axis=0)

# Get yearly player stats

### Keep all player data

In [172]:
season_play = active_player.groupby(["season", "codes", "name", "height", "weight", "sub_pos"])[["sp","pts",'fg', 'fga', 'fg3', 'fg3a', 'ft', 'fta',
       'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf']].mean().round(1).reset_index()

In [173]:
season_play['tov'] = -season_play['tov']
season_play['pf'] = -season_play['pf']

In [174]:
season_play.to_excel("season_player.xlsx")

### Calculate Min and Max for each numeric value, for each position (BC and FC are averaged) and for each year

##### Version without normalization

In [175]:
agg_df['tov'] = -agg_df['tov']
agg_df['pf'] = -agg_df['pf']

In [176]:
min_stats = agg_df.groupby(["season", "sub_pos"])[["height", "weight", "sp","pts",'fg', 'fga', 'fg3', 'fg3a', 'ft', 'fta',
       'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf']].min().reset_index()

In [177]:
min_stats["type"] = "min"

In [179]:
max_stats = agg_df.groupby(["season", "sub_pos"])[["height", "weight", "sp","pts",'fg', 'fga', 'fg3', 'fg3a', 'ft', 'fta',
       'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf']].max().reset_index()

In [180]:
max_stats["type"] = "max"

In [181]:
min_max_stats = pd.concat([min_stats, max_stats])

In [182]:
min_max_stats.to_excel("min_max_stats.xlsx")

In [183]:
for stat in num_cols:
    mm = (
        min_max_stats
        .pivot_table(
            index=["season", "sub_pos"],
            columns="type",
            values=stat
        )
        .reset_index()
        .rename(columns={"min": f"{stat}_min", "max": f"{stat}_max"})
    )

    agg_df = agg_df.merge(
        mm,
        left_on=["season", "sub_pos"],
        right_on=["season", "sub_pos"],
        how="left"
    )

    agg_df[f"{stat}_scaled"] = (
        (agg_df[stat] - agg_df[f"{stat}_min"])
        / (agg_df[f"{stat}_max"] - agg_df[f"{stat}_min"])
    ).round(3)

    agg_df = agg_df.drop(columns=[f"{stat}_min", f"{stat}_max"])


In [184]:
scaled_data = agg_df[['game_code', 'team', 'abrev', 'field','date', 'season','sub_pos', 'height_scaled', 'weight_scaled', 'sp_scaled',
                      'pts_scaled','fg_scaled', 'fga_scaled', 'fg3_scaled', 'fg3a_scaled', 'ft_scaled','fta_scaled', 'orb_scaled', 
                      'drb_scaled', 'ast_scaled', 'stl_scaled','blk_scaled', 'tov_scaled', 'pf_scaled','score', 'OT', 'target']]

In [185]:
scaled_data.to_excel("scaled_games.xlsx")

In [186]:
pos_dict = {1:"PG", 2:"SG", 3:"SF", 4:"PF", 5:"C"}

In [187]:
scaled_data['sub_pos'] = scaled_data['sub_pos'].apply(lambda x: pos_dict.get(x, x))

C:\Users\lione\AppData\Local\Temp\ipykernel_14652\4147159619.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scaled_data['sub_pos'] = scaled_data['sub_pos'].apply(lambda x: pos_dict.get(x, x))


In [189]:
scaled_data = scaled_data.rename(columns=lambda c: c.replace("_scaled", ""))

In [190]:
# colonnes fixes à garder
base_cols = ["game_code"]

# stats numériques à transformer
num_cols = ['height', 'weight', 'sp', 'pts', 'fg', 'fga', 'fg3', 'fg3a', 'ft',
       'fta', 'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf']


# ===== 1. pivot home / away =====
piv_field = (
    scaled_data
    .pivot_table(
        index=base_cols + ["sub_pos"],
        columns="field",
        values=num_cols,
        aggfunc="sum",
        fill_value=0      # BC / FC absents = 0
    )
)

# ===== 2. home - away =====
diff = piv_field.xs("Home", level=1, axis=1) \
                .sub(piv_field.xs("Away", level=1, axis=1), fill_value=0)

# ===== 3. pivot sub_pos en colonnes =====
final = (
    diff
    .reset_index()
    .pivot(
        index=base_cols,
        columns="sub_pos",
        values=num_cols
    )
)

# ===== 4. renommage des colonnes =====
final.columns = [
    f"{pos}_{stat}"
    for stat, pos in final.columns
]

final = final.reset_index()


In [191]:
match_info = (
    scaled_data[
        ["game_code", "season", "date", "score", "OT", "target"]
    ]
    .drop_duplicates("game_code")
)

In [192]:
final = final.merge(match_info, on="game_code", how="left")

In [194]:
final = final.sort_values(by=["date", "game_code"]).reset_index(drop=True)

##### Version with normalization

In [198]:
agg_df_time = agg_df_time.drop("sp", axis=1)

In [199]:
agg_df_time['tov'] = -agg_df_time['tov']
agg_df_time['pf'] = -agg_df_time['pf']

In [200]:
min_stats = agg_df_time.groupby(["season", "sub_pos"])[["height", "weight", "pts",'fg', 'fga', 'fg3', 'fg3a', 'ft', 'fta',
       'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf']].min().reset_index()

In [201]:
min_stats["type"] = "min"

In [203]:
max_stats = agg_df_time.groupby(["season", "sub_pos"])[["height", "weight", "pts",'fg', 'fga', 'fg3', 'fg3a', 'ft', 'fta',
       'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf']].max().reset_index()

In [204]:
max_stats["type"] = "max"

In [205]:
min_max_norm = pd.concat([min_stats, max_stats])

In [206]:
min_max_norm.to_excel("min_max_norm.xlsx")

In [208]:
num_cols = ['height', 'weight', 'pts', 'fg', 'fga', 'fg3', 'fg3a', 'ft',
       'fta', 'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf']

In [209]:
for stat in num_cols:
    mm = (
        min_max_norm
        .pivot_table(
            index=["season", "sub_pos"],
            columns="type",
            values=stat
        )
        .reset_index()
        .rename(columns={"min": f"{stat}_min", "max": f"{stat}_max"})
    )

    agg_df_time = agg_df_time.merge(
        mm,
        left_on=["season", "sub_pos"],
        right_on=["season", "sub_pos"],
        how="left"
    )

    agg_df_time[f"{stat}_scaled"] = (
        (agg_df_time[stat] - agg_df_time[f"{stat}_min"])
        / (agg_df_time[f"{stat}_max"] - agg_df_time[f"{stat}_min"])
    ).round(3)

    agg_df_time = agg_df_time.drop(columns=[f"{stat}_min", f"{stat}_max"])

In [211]:
scaled_data_norm = agg_df_time[['game_code', 'team', 'abrev', 'field','date', 'season','sub_pos', 'height_scaled', 'weight_scaled',
                      'pts_scaled','fg_scaled', 'fga_scaled', 'fg3_scaled', 'fg3a_scaled', 'ft_scaled','fta_scaled', 'orb_scaled', 
                      'drb_scaled', 'ast_scaled', 'stl_scaled','blk_scaled', 'tov_scaled', 'pf_scaled','score', 'OT', 'target']]

In [212]:
scaled_data_norm.to_excel("scaled_games_norm.xlsx")

In [213]:
pos_dict = {1:"PG", 2:"SG", 3:"SF", 4:"PF", 5:"C"}

In [214]:
scaled_data_norm['sub_pos'] = scaled_data_norm['sub_pos'].apply(lambda x: pos_dict.get(x, x))

C:\Users\lione\AppData\Local\Temp\ipykernel_14652\1867450984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scaled_data_norm['sub_pos'] = scaled_data_norm['sub_pos'].apply(lambda x: pos_dict.get(x, x))


In [215]:
scaled_data_norm = scaled_data_norm.rename(columns=lambda c: c.replace("_scaled", ""))

In [216]:
# colonnes fixes à garder
base_cols = ["game_code"]

# stats numériques à transformer
num_cols = ['height', 'weight', 'pts', 'fg', 'fga', 'fg3', 'fg3a', 'ft',
       'fta', 'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf']


# Pivot with home and away 
piv_field_norm = (
    scaled_data_norm
    .pivot_table(
        index=base_cols + ["sub_pos"],
        columns="field",
        values=num_cols,
        aggfunc="sum",
        fill_value=0      # for BC / FC = 0
    )
)

# For each game, each numeric data, make the difference between home and away (.xs() reach the level 1 of the multi-index created above)
diff_norm = piv_field_norm.xs("Home", level=1, axis=1) \
                .sub(piv_field_norm.xs("Away", level=1, axis=1), fill_value=0)

# Pivot sub_pos in columns
final_norm = (
    diff_norm
    .reset_index()
    .pivot(
        index=base_cols,
        columns="sub_pos",
        values=num_cols
    )
)

# Rename columns
final_norm.columns = [
    f"{pos}_{stat}"
    for stat, pos in final_norm.columns
]

final_norm = final_norm.reset_index()


In [217]:
match_info = (
    scaled_data_norm[
        ["game_code", "season", "date", "score", "OT", "target"]
    ]
    .drop_duplicates("game_code")
)

In [218]:
final_norm = final_norm.merge(match_info, on="game_code", how="left")

In [219]:
final_norm = final_norm.sort_values(by=["date", "game_code"]).reset_index(drop=True)

In [229]:
# OT means there have been overtime so we put 0 for the score difference

In [227]:
final_norm["diff"] = final_norm.apply(lambda x: 0 if pd.notna(x["OT"]) else int(x["score"].split("-")[1]) - int(x["score"].split("-")[0]), axis=1)

In [228]:
final_norm.to_excel("final_ML_data.xlsx")